In [1]:
import pandas as pd
import numpy as np
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl

In [2]:
os.chdir('G:\\\\Drives compartilhados\\\\República.org\\\\4. Equipes\\\\Dados e Comunicação\\\\DADOS\\\\415 - Repositório de Dados\\\\Repositório Local')


Chamando as bases

In [3]:
df18 = pd.read_excel('MUNIC//Base_MUNIC_2018.xlsx', sheet_name='Educação', usecols=['Cod Municipio','MEDU21'])
df18

,Cod Municipio,MEDU21
0,1100015,Não
1,1100023,Sim
2,1100031,Não
3,1100049,Não
4,1100056,Sim
...,...,...
5565,5222005,Não
5566,5222054,Não
5567,5222203,Não
5568,5222302,Não


In [4]:
df21 = pd.read_excel('MUNIC//Base_MUNIC_2021.xlsx', sheet_name='Educação', usecols=['CodMun','Medu21'])
df21

,CodMun,Medu21
0,1100015,Não
1,1100023,Não
2,1100031,Não
3,1100049,Sim
4,1100056,Não
...,...,...
5565,5222005,Sim
5566,5222054,Não
5567,5222203,Sim
5568,5222302,Não


In [5]:
uf = pd.read_excel('MUNIC//Base_MUNIC_2021.xlsx', sheet_name = 'Variáveis externas', usecols=[0,1,2,3])
uf 

,CodMun,UF,Cod UF,Mun
0,1100015,RO,11,Alta Floresta DOeste
1,1100023,RO,11,Ariquemes
2,1100031,RO,11,Cabixi
3,1100049,RO,11,Cacoal
4,1100056,RO,11,Cerejeiras
...,...,...,...,...
5565,5222005,GO,52,Vianópolis
5566,5222054,GO,52,Vicentinópolis
5567,5222203,GO,52,Vila Boa
5568,5222302,GO,52,Vila Propício


Renomeando as colunas

In [6]:
df18= df18.rename(columns={'Cod Municipio':'id_municipio',
                        'MEDU21':'plano_carreira_vigente'}) 

In [7]:
df21= df21.rename(columns={'CodMun':'id_municipio',
                           'Medu21':'plano_carreira_vigente'}) 

In [8]:
##adicionando a coluna de 'ano' em cada um dos dfs
df18['ano']=2018
df21['ano']=2021 

In [9]:
df = pd.concat([df18,df21]) #juntando os dataframes

In [10]:
df

,id_municipio,plano_carreira_vigente,ano
0,1100015,Não,2018
1,1100023,Sim,2018
2,1100031,Não,2018
3,1100049,Não,2018
4,1100056,Sim,2018
...,...,...,...
5565,5222005,Sim,2021
5566,5222054,Não,2021
5567,5222203,Sim,2021
5568,5222302,Não,2021


In [11]:
x= uf.pivot_table(columns=('CodMun', 'UF','Cod UF', 'Mun'), aggfunc='size')


In [12]:
uf = pd.DataFrame(x).reset_index()[['CodMun', 'UF','Cod UF', 'Mun']]

In [13]:
df = df.merge(uf, right_on='CodMun',left_on='id_municipio') #adicionando siglas e nome das UFs e dos municipios


In [14]:
df = df.drop(['CodMun'], axis=1) #eliminando coluna repetida

In [15]:
df= df.rename(columns={'UF':'sigla_uf',
                'Cod UF':'cod_uf',
                'Mun':'nome_municipio'}) #padronizando

In [26]:
A = df[df['plano_carreira_vigente']=='Recusa']
A

,ano,sigla_uf,id_municipio,nome_municipio,plano_carreira_vigente
495,2021,PA,1505908,Porto de Moz,Recusa
972,2018,MA,2102101,Brejo,Recusa
9119,2021,SC,4216602,São José,Recusa


In [17]:
df['plano_carreira_vigente']=np.where(df['plano_carreira_vigente']=='Não informou','Sem dados',df['plano_carreira_vigente'])

In [19]:
df = df[['ano','sigla_uf','id_municipio','nome_municipio','plano_carreira_vigente']] #organizando as colunas


In [20]:
df

,ano,sigla_uf,id_municipio,nome_municipio,plano_carreira_vigente
0,2018,RO,1100015,Alta Floresta DOeste,Não
1,2021,RO,1100015,Alta Floresta DOeste,Não
2,2018,RO,1100023,Ariquemes,Sim
3,2021,RO,1100023,Ariquemes,Não
4,2018,RO,1100031,Cabixi,Não
...,...,...,...,...,...
11135,2021,GO,5222203,Vila Boa,Sim
11136,2018,GO,5222302,Vila Propício,Não
11137,2021,GO,5222302,Vila Propício,Não
11138,2018,DF,5300108,Brasília,Sim


Subindo para o GBQ

In [21]:
client = bigquery.Client()
dataset_ref = client.dataset('estrutura_organizacional_carreiras')

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11140 entries, 0 to 11139
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   ano                     11140 non-null  int64 
 1   sigla_uf                11140 non-null  object
 2   id_municipio            11140 non-null  int64 
 3   nome_municipio          11140 non-null  object
 4   plano_carreira_vigente  11140 non-null  object
dtypes: int64(2), object(3)
memory usage: 522.2+ KB


In [23]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano da apuração daquele dado'),
        bigquery.SchemaField('sigla_uf','STRING',description='Sigla da UF'),
        bigquery.SchemaField('id_municipio','INTEGER',description='Identificador do município pelo IBGE'),
        bigquery.SchemaField('nome_municipio','STRING',description='Nome do município da observação'),       
        bigquery.SchemaField('plano_carreira_vigente','STRING',description='Se existe plano de carreira vigente para os profissionais da educação não docentes')]


In [24]:
table_ref = dataset_ref.table('MUNIC_existencia_plano_carreira_exceto_docentes')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df,table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=efd1866a-550a-4858-b272-da966b96363b>